# Diego Francisco Monroy Figueroa
## Aprendizaje por Refuerzo
### mon23318@guvg.edu.gt
### Laboratorio #10

1. Objetivos y mecánicas del juego (Galaxian):
  
  • Objetivo principal del juego:

    - El objetivo principal de Galaxian es controlar una nave espacial (el jugador) y disparar a los enemigos que vienen en oleadas desde la parte superior de la pantalla. El jugador debe evitar ser alcanzado por los disparos enemigos y destruir todas las naves enemigas para avanzar al siguiente nivel.

  • Mecánica del juego:

    - El jugador controla la nave que se desplaza horizontalmente en la parte inferior de la pantalla, disparando proyectiles hacia las naves enemigas.
    - Las naves enemigas se mueven en patrones predecibles pero se vuelven más difíciles a medida que el juego progresa.
    - El jugador tiene vidas limitadas, y perder todas las vidas termina el juego.

  • Puntuación:

    - El puntaje se determina por la cantidad de naves enemigas destruidas. Cada nave enemiga tiene un valor de puntuación y se suman conforme se destruyen.
    - También pueden aparecer bonificaciones adicionales, como una nave extra, que aumenta el puntaje del jugador.


2. Definición del estado del entorno:

En ALE/Galaxian-v5, el estado del entorno se refiere a la representación de la situación actual del juego, que se pasa al agente (la política que el agente usa para tomar decisiones).

  • Estado del entorno en Galaxian:

    - El estado generalmente se representa como una imagen de 240x256 píxeles en color RGB que muestra:

      o La nave del jugador en la parte inferior de la pantalla.
      
      o Las naves enemigas que están posicionadas en la parte superior de la pantalla.
      
      o Disparos tanto del jugador como de los enemigos.
      
      o Otras posibles características del entorno, como bonificaciones.

El agente observa estas imágenes de la pantalla y usa esta información para decidir qué acción tomar en cada momento del juego.


3. Acciones disponibles:

En Galaxian, las acciones disponibles para el agente (el jugador) se limitan principalmente al movimiento de la nave y el disparo:

  • Acciones disponibles:
    
    - Mover a la izquierda (acción 0): Desplaza la nave del jugador hacia la izquierda de la pantalla.
    - Mover a la derecha (acción 1): Desplaza la nave del jugador hacia la derecha de la pantalla.
    - Disparar (acción 2): La nave dispara un proyectil hacia los enemigos.

# Librerías

In [ ]:
import gymnasium as gym
import numpy as np
import imageio
from datetime import datetime
from pathlib import Path
import ale_py
import shimmy


# Entorno y grabación de video

In [ ]:
# Crear entorno con render RGB para capturar frames
env = gym.make("ALE/Galaxian-v5", render_mode="rgb_array")
n_actions = env.action_space.n
print("Acciones disponibles:", n_actions)

def run_episode(policy_fn, max_steps=100000):
    #Ejecuta un episodio y devuelve (total_reward, frames)
    frames = []
    obs, info = env.reset()
    done = False
    truncated = False
    total_reward = 0.0
    steps = 0

    while not (done or truncated):
        action = policy_fn(obs)
        obs, reward, done, truncated, info = env.step(action)
        frame = env.render()
        frames.append(frame)
        total_reward += float(reward)
        steps += 1
        if steps >= max_steps:
            break
    return total_reward, frames

def save_video(frames, score, email="mon23318", fps=30):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    name = f"{email}_{ts}_{int(score)}.mp4"
    outdir = Path("videos")
    outdir.mkdir(parents=True, exist_ok=True)
    path = outdir / name
    imageio.mimsave(path, frames, fps=fps)
    print(f"  Video guardado en: {path}")
    return str(path)

def record_episode(policy):
    """
    Esta función recibe una política y graba un episodio del juego.
    Devuelve el archivo de video guardado en formato MP4, así como la recompensa (score) y los frames.
    """
    score, frames = run_episode(policy)  # Obtener el score y los frames del episodio
    video_path = save_video(frames, score)  # Guardar el video con el nombre adecuado
    return score, frames  # Devolver score y frames



Acciones disponibles: 6


# Implementación Política Aleatoria (temporalmente)

In [ ]:
def random_policy(_obs):
    return env.action_space.sample()



# Main

In [ ]:
# Número de episodios a ejecutar
n_episodes = 1
total_scores = []

for i in range(n_episodes):
    print(f"🎮 Ejecutando episodio {i + 1}/{n_episodes}...")
    score, frames = record_episode(random_policy)  # Usar la política aleatoria
    total_scores.append(score)  # Guardar la recompensa en la lista
    print(f"🎮 Episodio {i + 1}: Score: {score}")

# Resumen de las recompensas obtenidas
print(f"\nPromedio de las recompensas de los {n_episodes} episodios: {np.mean(total_scores)}")



🎮 Ejecutando episodio 1/1...


  Video guardado en: videos/mon23318_20251029_045151_1010.mp4
🎮 Episodio 1: Score: 1010.0

Promedio de las recompensas de los 1 episodios: 1010.0


# Resultados

In [ ]:
import os

video_dir = Path("videos")
if os.path.exists(video_dir):
    video_files = list(video_dir.glob("*.mp4"))
    print(f"Videos guardados en: {video_files}")
else:
    print("No se encontraron videos. Asegúrate de ejecutar los episodios primero.")



Videos guardados en: [PosixPath('videos/mon23318_20251029_042401_1330.mp4'), PosixPath('videos/mon23318_20251029_041238_1340.mp4'), PosixPath('videos/mon23318_20251029_041158_1460.mp4'), PosixPath('videos/mon23318_20251029_042359_930.mp4'), PosixPath('videos/mon23318_20251029_042403_1060.mp4'), PosixPath('videos/mon23318_20251029_045151_1010.mp4')]


In [ ]:
!pip install gymnasium[atari,accept-rom-license] ale-py shimmy imageio imageio-ffmpeg


In [ ]:
!pip install AutoROM
!AutoROM --accept-license

import sys, subprocess
subprocess.run([sys.executable, "-m", "AutoROM", "--accept-license"], check=True)